In [101]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [102]:
from datetime import datetime

In [103]:
results = []

month_map = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09',
             'Oct':'10','Nov':'11','Dec':'12'}

data_history = {
    "Date": [],
    "source": [],
    "sentiment":[]
}
#stocknewsapi has LLM -GPT4 analyzed sentiment --> which categorizes the news into the buckets positive, negative, neutral
for i in range (1,100):#max10000 data 
    api_url = "https://stocknewsapi.com/api/v1?tickers=AMZN&data=yeartodate&items=100&page="+ str(i) +"&token=metobjbevvotlrizynavabhyisukldmxcxsdwgrf"
    response = requests.get(api_url)
    response_data = response.json()
    for rd in response_data["data"]:
        date_text = rd["date"].split(' ')
        dt = date_text[3] + month_map[date_text[2]] + date_text[1]
        data_history["Date"].append(pd.Timestamp(dt))
        data_history["source"].append(rd["source_name"])
        data_history["sentiment"].append(rd["sentiment"])
    results += response_data["data"]
import pandas as pd
df = pd.DataFrame(data_history)
print(df)

           Date                     source sentiment
0    2023-08-07              InvestorPlace  Positive
1    2023-08-07              InvestorPlace  Positive
2    2023-08-07               Fox Business  Negative
3    2023-08-07                    Reuters  Positive
4    2023-08-07               Market Watch  Negative
...         ...                        ...       ...
9895 2021-07-15              Business Wire   Neutral
9896 2021-07-15  Zacks Investment Research  Positive
9897 2021-07-15               CNN Business  Negative
9898 2021-07-15            The Motley Fool   Neutral
9899 2021-07-15            The Motley Fool  Positive

[9900 rows x 3 columns]


In [104]:
options_data = pd.read_csv('/Users/manaswiniswamy/Desktop/Book1-Options.csv',parse_dates=True,index_col='Date')
options_data

,call_volume,call_percentage,put_volume,put_percentage
Date,,,,
2022-08-04,1893592,0.57,1401144,0.43
2022-08-05,1339990,0.58,978214,0.42
2022-08-08,901492,0.54,761950,0.46
2022-08-09,605572,0.52,558114,0.48
2022-08-10,843904,0.52,763680,0.48
...,...,...,...,...
2023-07-31,491712,0.61,315620,0.39
2023-08-01,472038,0.63,277290,0.37
2023-08-02,749284,0.62,450228,0.38


In [105]:
# fetch the derivatives data (this decides the volatility and the vix index)--> will integrate it today

In [106]:
df['sentiment'] = df['sentiment'].map({'Positive': 1, 'Negative': -1, 'Neutral':0})
df

,Date,source,sentiment
0,2023-08-07,InvestorPlace,1
1,2023-08-07,InvestorPlace,1
2,2023-08-07,Fox Business,-1
3,2023-08-07,Reuters,1
4,2023-08-07,Market Watch,-1
...,...,...,...
9895,2021-07-15,Business Wire,0
9896,2021-07-15,Zacks Investment Research,1
9897,2021-07-15,CNN Business,-1
9898,2021-07-15,The Motley Fool,0


In [107]:
stock_data = pd.read_csv('/Users/manaswiniswamy/Downloads/AMZN.csv', parse_dates=True, index_col='Date')

In [108]:
stock_data['200_day_avg'] = stock_data['Close'].rolling(window=200).mean()
stock_data['50_day_avg'] = stock_data['Close'].rolling(window=50).mean()
stock_data['10_day_avg'] = stock_data['Close'].rolling(window=10).mean()

In [109]:
stock_data.dropna(inplace=True)

In [110]:
stock_data['Trend'] = np.where(stock_data['Close'].shift(-1) > stock_data['Close'], 1, 0)

In [111]:
stock_data=stock_data.merge(df,how="inner",on="Date")
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,200_day_avg,50_day_avg,10_day_avg,Trend,source,sentiment
0,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,InvestorPlace,1
1,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0
2,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0
3,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,CNN Business,-1
4,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Zacks Investment Research,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Market Watch,0
969,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,WSJ,0
970,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,TechXplore,1
971,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Proactive Investors,1


In [112]:
stock_data=stock_data.merge(options_data,how="inner",on="Date")
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,200_day_avg,50_day_avg,10_day_avg,Trend,source,sentiment,call_volume,call_percentage,put_volume,put_percentage
0,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,InvestorPlace,1,1340454,0.68,642056,0.32
1,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0,1340454,0.68,642056,0.32
2,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0,1340454,0.68,642056,0.32
3,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,CNN Business,-1,1340454,0.68,642056,0.32
4,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Zacks Investment Research,0,1340454,0.68,642056,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Market Watch,0,3631558,0.65,1964956,0.35
969,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,WSJ,0,3631558,0.65,1964956,0.35
970,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,TechXplore,1,3631558,0.65,1964956,0.35
971,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Proactive Investors,1,3631558,0.65,1964956,0.35


In [113]:
X = stock_data[['200_day_avg', '50_day_avg', '10_day_avg','sentiment','call_volume','call_percentage','put_volume','put_percentage']].values
y = stock_data['Trend'].values

In [114]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [116]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [117]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [118]:
model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_split=0.1)

Epoch 1/2000
22/22 [==============================] - 0s 5ms/step - loss: 0.6813 - accuracy: 0.5857 - val_loss: 0.6657 - val_accuracy: 0.6282
Epoch 2/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6620 - accuracy: 0.6043 - val_loss: 0.6538 - val_accuracy: 0.6282
Epoch 3/2000
22/22 [==============================] - 0s 2ms/step - loss: 0.6475 - accuracy: 0.6314 - val_loss: 0.6390 - val_accuracy: 0.6667
Epoch 4/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6330 - accuracy: 0.6714 - val_loss: 0.6234 - val_accuracy: 0.7051
Epoch 5/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6181 - accuracy: 0.6743 - val_loss: 0.6060 - val_accuracy: 0.6795
Epoch 6/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.6044 - accuracy: 0.7029 - val_loss: 0.5962 - val_accuracy: 0.6795
Epoch 7/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.5948 - accuracy: 0.6829 - val_loss: 0.5863 - val_accuracy: 0.6795
Epoch 

22/22 [==============================] - 0s 1ms/step - loss: 0.3365 - accuracy: 0.8443 - val_loss: 0.3447 - val_accuracy: 0.7949
Epoch 59/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3398 - accuracy: 0.8243 - val_loss: 0.3428 - val_accuracy: 0.8077
Epoch 60/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3356 - accuracy: 0.8271 - val_loss: 0.3419 - val_accuracy: 0.7949
Epoch 61/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3330 - accuracy: 0.8286 - val_loss: 0.3537 - val_accuracy: 0.8590
Epoch 62/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3282 - accuracy: 0.8386 - val_loss: 0.3449 - val_accuracy: 0.8846
Epoch 63/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3272 - accuracy: 0.8471 - val_loss: 0.3341 - val_accuracy: 0.8462
Epoch 64/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3257 - accuracy: 0.8529 - val_loss: 0.3375 - val_accuracy: 0.8718
Epoch 65/2000

22/22 [==============================] - 0s 1ms/step - loss: 0.2581 - accuracy: 0.8714 - val_loss: 0.3285 - val_accuracy: 0.8077
Epoch 116/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2479 - accuracy: 0.8871 - val_loss: 0.2909 - val_accuracy: 0.8590
Epoch 117/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2433 - accuracy: 0.8843 - val_loss: 0.2928 - val_accuracy: 0.8462
Epoch 118/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2407 - accuracy: 0.8814 - val_loss: 0.2971 - val_accuracy: 0.8462
Epoch 119/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2392 - accuracy: 0.8843 - val_loss: 0.2831 - val_accuracy: 0.8462
Epoch 120/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2374 - accuracy: 0.8786 - val_loss: 0.2678 - val_accuracy: 0.8718
Epoch 121/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.2384 - accuracy: 0.8943 - val_loss: 0.2621 - val_accuracy: 0.8846
Epoch 1

Epoch 172/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1814 - accuracy: 0.9171 - val_loss: 0.2644 - val_accuracy: 0.8205
Epoch 173/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1798 - accuracy: 0.9229 - val_loss: 0.2162 - val_accuracy: 0.9103
Epoch 174/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1791 - accuracy: 0.9329 - val_loss: 0.2081 - val_accuracy: 0.9231
Epoch 175/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1712 - accuracy: 0.9257 - val_loss: 0.1946 - val_accuracy: 0.9359
Epoch 176/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1709 - accuracy: 0.9357 - val_loss: 0.2091 - val_accuracy: 0.9231
Epoch 177/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1719 - accuracy: 0.9257 - val_loss: 0.2270 - val_accuracy: 0.8974
Epoch 178/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1764 - accuracy: 0.9329 - val_loss: 0.2531 - val_accuracy:

Epoch 229/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1227 - accuracy: 0.9557 - val_loss: 0.1690 - val_accuracy: 0.9359
Epoch 230/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1246 - accuracy: 0.9500 - val_loss: 0.1477 - val_accuracy: 0.9744
Epoch 231/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1276 - accuracy: 0.9543 - val_loss: 0.1774 - val_accuracy: 0.9231
Epoch 232/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1232 - accuracy: 0.9557 - val_loss: 0.1711 - val_accuracy: 0.9231
Epoch 233/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1195 - accuracy: 0.9486 - val_loss: 0.1452 - val_accuracy: 0.9744
Epoch 234/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1227 - accuracy: 0.9543 - val_loss: 0.1686 - val_accuracy: 0.9359
Epoch 235/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.1226 - accuracy: 0.9529 - val_loss: 0.1511 - val_accuracy:

...

22/22 [==============================] - 0s 1ms/step - loss: 8.1507e-09 - accuracy: 1.0000 - val_loss: 2.5954e-08 - val_accuracy: 1.0000
Epoch 1749/2000
22/22 [==============================] - 0s 1ms/step - loss: 8.2428e-09 - accuracy: 1.0000 - val_loss: 3.6372e-08 - val_accuracy: 1.0000
Epoch 1750/2000
22/22 [==============================] - 0s 1ms/step - loss: 8.1668e-09 - accuracy: 1.0000 - val_loss: 3.3898e-08 - val_accuracy: 1.0000
Epoch 1751/2000
22/22 [==============================] - 0s 1ms/step - loss: 8.2422e-09 - accuracy: 1.0000 - val_loss: 3.7012e-08 - val_accuracy: 1.0000
Epoch 1752/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.9914e-09 - accuracy: 1.0000 - val_loss: 3.0724e-08 - val_accuracy: 1.0000
Epoch 1753/2000
22/22 [==============================] - 0s 1ms/step - loss: 8.1176e-09 - accuracy: 1.0000 - val_loss: 3.4656e-08 - val_accuracy: 1.0000
Epoch 1754/2000
22/22 [==============================] - 0s 1ms/step - loss: 8.5472e-09 - accuracy

22/22 [==============================] - 0s 1ms/step - loss: 6.5748e-09 - accuracy: 1.0000 - val_loss: 3.3192e-08 - val_accuracy: 1.0000
Epoch 1802/2000
22/22 [==============================] - 0s 1ms/step - loss: 9.0577e-09 - accuracy: 1.0000 - val_loss: 2.9779e-08 - val_accuracy: 1.0000
Epoch 1803/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.2908e-09 - accuracy: 1.0000 - val_loss: 2.4020e-08 - val_accuracy: 1.0000
Epoch 1804/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.2976e-09 - accuracy: 1.0000 - val_loss: 2.5641e-08 - val_accuracy: 1.0000
Epoch 1805/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.5143e-09 - accuracy: 1.0000 - val_loss: 2.9964e-08 - val_accuracy: 1.0000
Epoch 1806/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.8739e-09 - accuracy: 1.0000 - val_loss: 3.1975e-08 - val_accuracy: 1.0000
Epoch 1807/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.9428e-09 - accuracy

22/22 [==============================] - 0s 1ms/step - loss: 7.1578e-09 - accuracy: 1.0000 - val_loss: 1.9202e-08 - val_accuracy: 1.0000
Epoch 1855/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.5711e-09 - accuracy: 1.0000 - val_loss: 3.5012e-08 - val_accuracy: 1.0000
Epoch 1856/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.1609e-09 - accuracy: 1.0000 - val_loss: 1.7434e-08 - val_accuracy: 1.0000
Epoch 1857/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.0204e-09 - accuracy: 1.0000 - val_loss: 2.5821e-08 - val_accuracy: 1.0000
Epoch 1858/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.0270e-09 - accuracy: 1.0000 - val_loss: 1.6848e-08 - val_accuracy: 1.0000
Epoch 1859/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.5554e-09 - accuracy: 1.0000 - val_loss: 2.4592e-08 - val_accuracy: 1.0000
Epoch 1860/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.2799e-09 - accuracy

Epoch 1908/2000
22/22 [==============================] - 0s 1ms/step - loss: 8.0689e-06 - accuracy: 1.0000 - val_loss: 1.5289e-05 - val_accuracy: 1.0000
Epoch 1909/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.8091e-06 - accuracy: 1.0000 - val_loss: 1.4972e-05 - val_accuracy: 1.0000
Epoch 1910/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.5562e-06 - accuracy: 1.0000 - val_loss: 1.4636e-05 - val_accuracy: 1.0000
Epoch 1911/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.3327e-06 - accuracy: 1.0000 - val_loss: 1.4385e-05 - val_accuracy: 1.0000
Epoch 1912/2000
22/22 [==============================] - 0s 1ms/step - loss: 7.1214e-06 - accuracy: 1.0000 - val_loss: 1.4007e-05 - val_accuracy: 1.0000
Epoch 1913/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.9122e-06 - accuracy: 1.0000 - val_loss: 1.3739e-05 - val_accuracy: 1.0000
Epoch 1914/2000
22/22 [==============================] - 0s 1ms/step - loss: 6.718

22/22 [==============================] - 0s 1ms/step - loss: 2.7502e-06 - accuracy: 1.0000 - val_loss: 6.7129e-06 - val_accuracy: 1.0000
Epoch 1962/2000
22/22 [==============================] - 0s 1ms/step - loss: 2.7103e-06 - accuracy: 1.0000 - val_loss: 6.6126e-06 - val_accuracy: 1.0000
Epoch 1963/2000
22/22 [==============================] - 0s 1ms/step - loss: 2.6698e-06 - accuracy: 1.0000 - val_loss: 6.5292e-06 - val_accuracy: 1.0000
Epoch 1964/2000
22/22 [==============================] - 0s 1ms/step - loss: 2.6322e-06 - accuracy: 1.0000 - val_loss: 6.4682e-06 - val_accuracy: 1.0000
Epoch 1965/2000
22/22 [==============================] - 0s 1ms/step - loss: 2.5976e-06 - accuracy: 1.0000 - val_loss: 6.3643e-06 - val_accuracy: 1.0000
Epoch 1966/2000
22/22 [==============================] - 0s 1ms/step - loss: 2.5594e-06 - accuracy: 1.0000 - val_loss: 6.2851e-06 - val_accuracy: 1.0000
Epoch 1967/2000
22/22 [==============================] - 0s 1ms/step - loss: 2.5238e-06 - accuracy

In [119]:
loss, accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 867us/step - loss: 1.6500e-06 - accuracy: 1.0000


In [120]:
print(f"Test accuracy: {accuracy}")

Test accuracy: 1.0
